In [73]:
import wings
from wings.planner import Planner
from wings.component import Component
from wings.execution import Execution

In [74]:
SERVER = "http://localhost:9090/wings-portal"
INTERNAL_SERVER = SERVER

In [75]:
USERID = "admin"
PASSWORD = "admin123"
DOMAIN = "CaesarCypherParallel"

In [76]:
TESTS = [
    {
        "template": "CaesarCypherMap",
        "inputs": {
            "Document": "file:TwelveDaysOfChristmas.txt",
            "ShiftKey": 3,
            "numberOfChunks": 4
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "breakDoc": 1,
                            "Caesar": 4
                        },
                        "execute": True
                    }

                ]
            }
        }
    }
]

In [77]:
TESTS2 = [
    {
        "template": "Test_Spectral",
        "inputs": {
            "InputTS": "file:test_perfect_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt-psd": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "MTM": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "Welch": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "Periodogram": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Spectral",
        "inputs": {
            "InputTS": "file:test_noisy_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "wwz-psd": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "LombScargle": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Wavelet",
        "inputs": {
            "InputTS": "file:test_perfect_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt": 1,
                            "AR1sim-wave": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Wavelet",
        "inputs": {
            "InputTS": "file:test_noisy_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt": 1,
                            "AR1sim-wave": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    }
]

In [78]:
def getTemplateComponents(expansion):
    template = expansion["template"]
    components = {}
    for nodeid in template["Nodes"]:
        cvarId = template["Nodes"][nodeid]["componentVariable"]["binding"]["id"]
        cvar = cvarId.split("#")[-1]
        cvarnum = components.get(cvar, 0)
        cvarnum += 1
        components[cvar] = cvarnum
    return components

In [79]:
def matches_test_expansion(expansion, test_expansions):
    components = getTemplateComponents(expansion)
    print("> " + str(components))
    for test_expansion in test_expansions:
        if components == test_expansion["components"]:
            return test_expansion
    return None

In [80]:
api = wings.ApiClient(server=SERVER, username=USERID, password=PASSWORD)
api.set_domain(DOMAIN)

In [81]:
# Clean the Component Cache
# FIXME: Not cleaning right now to avoid data pollution while testing
# - Need a way to clean the test data after tests are done

#components = Component(api)
#components.increment_versions_clear_cache()

In [82]:
# Plan and check
planner = Planner(api)
for test in TESTS:
    print("Testing template: " + test['template'])
    test_expansions = test["outputs"]["planner"]["expansions"]

    print("> Planning workflow expansions")
    planner.set_template(test["template"])
    plan = planner.get_expansions(test["inputs"])
    if plan["success"]:
        expansions = plan["data"]["templates"]
        seed = plan["data"]["seed"]
        if len(test_expansions) != len(expansions):
            print(">> FAILED: Expected %d expanded workflows, got %d" % (len(test_expansions), len(expansions)))
            continue

        runids = []
        for expansion in expansions:
            print("> Testing expansion")
            test_expansion = matches_test_expansion(expansion, test_expansions)
            if test_expansion is not None:
                print(">> Expanded workflow components match expected")                
                if "execute" in test_expansion and test_expansion["execute"]:
                    print(">> Submitting for execution")
                    runid = planner.run_workflow(expansion, seed)
                    runids.append(runid)
            else:
                print(">> FAILED: Expanded workflow components do not match expected")
    else:
        print(">> FAILED: Planning failed")
        print(plan)


Testing template: CaesarCypherMap
> Planning workflow expansions
> Testing expansion
> {'breakDoc': 1, 'Caesar': 4}
>> Expanded workflow components match expected
>> Submitting for execution


In [83]:
execution_api = Execution(api)

for runid in runids:
    print("Checking " + runid)
    details = execution_api.get_run_details(runid)
    run_status = details["execution"]["runtimeInfo"]["status"]
    # run_status should be "SUCCESS", or "FAILURE".. if "RUNNING" or "QUEUED", keep waiting
    print(run_status) 

Checking CaesarCypherMap--71561b11-8704-4931-b4c1-42ed6705ee58
RUNNING


In [84]:
api.logout()